In [ ]:
from datetime import datetime, timedelta
import pytz
import math
import pylab as plt

import pandas as pd
import numpy as np

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

%load_ext autoreload

%matplotlib widget
from plotter import Plot
from filewriter import ExcelWriter as ex

from extractor import WeatherExtractor, Extractor, Period

from inversegreyboxmodel import Learner

import logging
logger = logging.getLogger('Twomes data extraction')
logger.setLevel(logging.NOTSET)



n_std_outliers = 3.0 # default for the multiplier of the the standard deviation; further out than this times the std, outliers are removed during preprocessing
up_intv = '5min' # the default upsampling interval that is used before interpolation is done
gap_n_intv = 11 # the default maximum number of consecutive NaNs to fill(one for each upsampling interval), i.e. valid measurement values (11+1)* 5 min = 1 hour apart apart will be bridget by interpolation, but not more
sampling_interval = '15min' # the default interval on which interpolation will be done during preprocessing
moving_horizon_duration_d = 7
required_columns_for_sanity = ['home_id', 'T_out_e_avg_C', 'irradiation_hor_avg_W_p_m2', 'T_in_avg_C', 'gas_sup_avg_W', 'e_remaining_heat_avg_W', 'interval_s']
        

sanity_fraction = 0.5

#location: center of Assendorp neighbourhood in Zwolle
lat, lon = 52.50655, 6.09961

#timezone: 
timezone_database = 'UTC'
timezone_homes = 'Europe/Amsterdam'

# TODO: get list of pseudonyms, (valid) dates and reference parameters from Excel file?  
# utimately: #min, max dates of the analysis

# # # Below, the maximum period for data collection
# first_day = pytz.timezone(timezone_homes).localize(datetime(2021, 10, 25))
# last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 5, 8))


# The full set of homes
homes = [803422, 805164, 809743, 811308, 815925, 817341, 822479, 829947, 830088, 831062, 839440, 845966, 845997, 846697, 857477, 864296, 873985, 879481, 881611, 886307, 895671, 897349, 899510]

# # A subset of homes
# homes = [803422, 805164, 809743]

# single home for virtual homes
# homes = [886307]

# Alternatively, you may want to test things only on a three week periode. This is a period with suitable weather and lots of homes with measurements.
first_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 3))
last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 31))


In [ ]:
%%time 
%autoreload 2
# get geospatially interpolated weather from KNMI
# for Twomes, the Weather for all all homes studies can be approached by a single location
# get the dataframe only once for all homes to save time
tz_knmi='Europe/Amsterdam'

df_weather = WeatherExtractor.get_interpolated_weather_nl(first_day, last_day, lat, lon, tz_knmi, timezone_homes, sampling_interval)

# plot temperature data
logger.setLevel(logging.NOTSET)
Plot.weather_and_other_temperatures('Weather in Assendorp, Zwolle', df_weather)

# N.B. The resulting figure below can be manipulated interactively; hover with mouse for tips & tricks

In [ ]:
#see more statisctics about the weather data
df_weather.describe(include='all')

In [ ]:
# %%time 
# # get interpolated data from the Twomes database and combine with weather data already obtained

# logger.setLevel(logging.INFO)

# df_data_homes = Extractor.get_preprocessed_homes_data(homes, first_day, last_day, timezone_database, timezone_homes,
#                                                       up_intv, gap_n_intv, sampling_interval, 
#                                                       required_columns_for_sanity,
#                                                       df_weather)
# filename_prefix = datetime.now().astimezone(pytz.timezone('Europe/Amsterdam')).replace(microsecond=0).isoformat().replace(":","")
# ex.write(df_data_homes, str('{0}-data_homes-{1}-{2}.xlsx'.format(filename_prefix, first_day.isoformat(),last_day.isoformat())))
# Extractor.write_home_data_to_csv(df_data_homes, str('{0}-data_homes-{1}-{2}.csv'.format(filename_prefix, first_day.isoformat(),last_day.isoformat())))

# logger.setLevel(logging.NOTSET)


In [ ]:
%%time 
# get interpolated data from the virtual homes and combine with weather data already obtained

logger.setLevel(logging.INFO)

homes = [
    60200, 
    120100, 
    150080, 
    150100, 
    200060, 
    300040, 
    400030, 
    600020 
]

df_data_homes = pd.DataFrame()
for home_id in homes:
    df_data_homes = pd.concat([df_data_homes, Extractor.get_virtual_home_data_csv(str('../data/virtualhome_P{0}.csv'.format(home_id)), timezone_homes)], axis=0)

logger.setLevel(logging.NOTSET)


In [ ]:
df_data_homes

In [ ]:
# present some sanity metrics for the extracted data
total_measurement_time = timedelta(seconds = int(df_data_homes['interval_s'].sum()))
print('Total measurement time: ', total_measurement_time)
sane_fraction = df_data_homes['sanity_frac'].mean()
print('Sane fraction measurement time: {:.2f}'.format(sane_fraction))
sane_measurement_time = total_measurement_time * sane_fraction
print('Sane  measurement time: ', sane_measurement_time)

#see more statisctics
df_data_homes.describe(include='all')

In [ ]:
# %%time 
# from tqdm import tqdm_notebook

# %autoreload 2
# filename_prefix = datetime.now().astimezone(pytz.timezone('Europe/Amsterdam')).replace(microsecond=0).isoformat().replace(":","")

# first_day = pytz.timezone(timezone_homes).localize(datetime(2021, 10, 25))
# last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 5, 8))

# df_rawdata = pd.DataFrame()
# home_iterator = tqdm_notebook(homes)

# for home_id in home_iterator:
#     # print('Processing ', home_id)
#     extractor = Extractor(home_id, Period(first_day, last_day))
#     df_rawdata = extractor.get_rawdata()
#     df_rawdata.describe(include='all')
#     Extractor.write_raw_data_to_csv(df_rawdata, str('{0}-rawdata_P{1}-{2}-{3}.csv'.format(filename_prefix, home_id, first_day.isoformat(),last_day.isoformat())))



In [ ]:
%%time 
%autoreload 2


sanity_fraction_analysis = sanity_fraction

# Use one of the lines below to set the moving horizon duration used for analysis 
# moving_horizon_duration_d_analysis = 14
moving_horizon_duration_d_analysis = moving_horizon_duration_d


# learn the model parameters and write rerults an intermediate results to excel files
df_results = Learner.learn_home_parameter_moving_horizon(df_data_homes, 
                                                         n_std_outliers, up_intv, gap_n_intv, sampling_interval, 
                                                         moving_horizon_duration_d_analysis, sanity_fraction_analysis,
                                                         hint_A_m2=None, ev_type=2)



In [ ]:
#show the results
df_results

In [ ]:
# plot temperature data of multiple homes from an array 
# # %autoreload 2
# for home_id in homes_to_analyze:
#     df_data_one_home = df_data_homes.loc[home_id]
#     Plot.weather_and_other_temperatures(home_id, df_data_one_home, [('T_in_avg_C','r'),('T_set_first_C','g')])

# # N.B. The resulting figure below can be manipulated interactively; hover with mouse for tips & tricks


In [ ]:
#plot a series of weeks for all homes
# plt.ioff()

# for home_id in homes_to_analyze:
#     df_data_one_home = df_data_homes.loc[home_id]
#     for moving_horizon_start in pd.date_range(start=first_day, end=first_day, inclusive='left', freq='7D'):
#         moving_horizon_end = min(first_day, moving_horizon_start + timedelta(days=7))
#         df_moving_horizon = df_data_one_home[moving_horizon_start:moving_horizon_end]
#         Plot.weather_and_other_temperatures(home_id, df_moving_horizon, ['indoor_temp_degC'])

# plt.ion()  
    

In [ ]:
# how to select data from a single home
# df_data_homes.loc[817341]